# Генерирование текста с помощью LSTM

В этом разделе мы посмотрим, как можно использовать рекуррентные нейронные сети для генерирования последовательностей данных. В качестве примера мы будем генерировать текст, однако представленные здесь методы можно распространить на любые последовательные данные: вы можете применить их к последовательности музыкальных нот и получить новую музыку или к последовательности данных, описывающих мазки кистью (например, записанных в процессе рисования художником на iPad), и сгенерировать картину мазок за мазком и т. д.
Генерирование последовательностей данных не ограничивается созданием художественных произведений. Этот прием с успехом используется для синтеза речи и генерирования диалогов для чат-ботов. Функция Smart Reply, представленная компанией Google в 2016 году и способная автоматически генерировать короткие ответы на электронные письма или текстовые сообщения, основана на подобных приемах.

Универсальный способ генерации последовательностей данных с применением методов глубокого обучения заключается в обучении сети (обычно рекуррентной или сверточной сети) для прогнозирования следующего токена или следующих нескольких токенов в последовательности, опираясь на предыдущие токены. Добавить сгенерированный вывод в конец предыдущих входных данных и повторить процесс много раз (рис. 8.1). Этот цикл позволяет генерировать последовательности произвольной длины, отражающие структуру данных, на которых обучалась модель.

В примере, представленном далее в этом разделе, мы возьмем слой LSTM, передадим ему строки длиной N символов, извлеченные из текстового корпуса, и обучим его предсказывать символ N + 1. На выходе модель будет возвращать вектор softmax с вероятностями для всех возможных символов: распределение вероятностей для следующего символа. Такой слой LSTM называется языковой нейронной моделью уровня символов.

Для генерации текста важную роль играет алгоритм выбора следующего символа. Наивное решение — жадный выбор, когда выбирается наиболее вероятный символ. Но такой подход приводит к получению в результате повторяющихся, предсказуемых строк, которые не выглядят связными предложениями. Намного интереснее подход, который делает порой неожиданный выбор, вводя случайную составляющую в процесс выбора из распределения вероятностей для следующего символа. Этот подход называется стохастическим выбором (как вы помните, слово стохастический в данном контексте является синонимом слова случайный). Таким образом, если символ e имеет вероятность 0,3 стать следующим символом, согласно модели мы выберем его в 30 % случаев. Обратите внимание на то, что жадный выбор тоже может использоваться для выбора из распределения вероятностей, когда какой-то символ имеет вероятность 1, а все остальные — вероятность 0.

Для управления величиной случайности в процессе выбора введем параметр, который назовем температурой softmax, характеризующий энтропию распределения вероятностей, используемую для выбора: она будет определять степень необычности или предсказуемости выбора следующего символа.

In [1]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5): 
    distribution = np.log(original_distribution) / temperature 
    distribution = np.exp(distribution) 
    return distribution / np.sum(distribution)

# original_distribution — это одномерный массив Numpy значений вероятностей, сумма которых должна быть равна 1

# Функция возвращает новую, взвешенную версию оригинального распределения. 
# Сумма вероятностей в новом распределении может получиться больше 1, поэтому разделим элементы вектора на сумму, 
# чтобы получить новое распределение

Чем выше температура, тем больше энтропия распределения вероятностей и тем более неожиданными и менее структурированными будут генерируемые данные. Чем меньше температура, тем меньше будет величина случайной составляющей и тем более предсказуемыми будут генерируемые данные

In [3]:
import tensorflow.keras
import numpy as np
path = tensorflow.keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt') #загрузка произведения ницше
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 4s 7us/step
Corpus length: 600901


Затем извлечем частично перекрывающиеся последовательности с длиной maxlen, выполним прямое кодирование и упакуем в трехмерный массив Numpy x с формой (последовательности, максимальная_длина, уникальные_символы). Одновременно подготовим массив y с соответствующими целями: векторы с символами, полученные прямым кодированием, которые следуют за каждой извлеченной последовательностью.

In [5]:
maxlen = 60 # Извлечение последовательностей по 60 символов
step = 3 # Новые последовательности выбираются через каждые 3 символа
sentences = [] # Хранение извлеченных последовательностей
next_chars = [] # Хранение целей (символов, следующих за последовательностями)
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text))) # Список уникальных символов в корпусе
print('Unique characters:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) # Словарь, отображающий уникальные символы в их индексы в списке «chars»
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences): 
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1 
        y[i, char_indices[next_chars[i]]] = 1 # Прямое кодирование символов в бинарные массивы

Number of sequences: 200281
Unique characters: 59
Vectorization...


### Конструирование сети
Эта сеть состоит из единственного слоя LSTM, за которым следует классификатор Dense с функцией softmax выбора из всех возможных символов. Но имейте в виду, что рекуррентные нейронные сети не единственный способ генерирования последовательностей данных; одномерные сверточные сети тоже показали превосходные результаты в решении этой задачи.

In [11]:
from tensorflow.keras import layers
model = tensorflow.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Имея обученную модель и фрагмент начального текста, можно сгенерировать новый текст, выполнив следующие пункты:

1. Извлечь из модели распределение вероятностей следующего символа для имеющегося на данный момент сгенерированного текста.

2. Выполнить взвешивание распределения с заданной температурой.

3. Выбрать следующий символ в соответствии с вновь взвешенным распределением вероятностей.

4. Добавить новый символ в конец текста.

In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Наконец, следующий цикл повторяет обучение и генерирует текст. Для начала сгенерируем текст, использовав разные температуры после каждой эпохи. Это позволит вам увидеть, как меняется генерируемый тест по мере схождения модели и как температура влияет на стратегию выбора.

In [13]:
import random
import sys
for epoch in range(1, 60): #обучение модели в течении 60 эпох
    print('epoch', epoch) 
    model.fit(x, y, batch_size=128, epochs=1) #выполнение одной итерации обучения
    
    start_index = random.randint(0, len(text) - maxlen - 1) #выбор случайного начального текста
    generated_text = text[start_index: start_index + maxlen] 
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]: #генерация текста для разных температур
        print('------ temperature:', temperature) 
        sys.stdout.write(generated_text) 
        for i in range(400): # Генерация 400 символов, начиная с начального текста 
            sampled = np.zeros((1, maxlen, len(chars))) 
            for t, char in enumerate(generated_text):  # Прямое кодирование символов, сгенерированных до сих пор
                sampled[0, t, char_indices[char]] = 1. 
            preds = model.predict(sampled, verbose=0)[0] #Выбор следующего символа
            next_index = sample(preds, temperature) #выбор индекса следующего символа
            next_char = chars[next_index] #следующий символ
            generated_text += next_char 
            generated_text = generated_text[1:] 
            sys.stdout.write(next_char)

epoch 1
Train on 200281 samples
200281/200281 [==============================] - 409s 2ms/sample - loss: 1.9628
--- Generating with seed: "thoven
is the intermediate event between an old mellow soul "
------ temperature: 0.2
thoven
is the intermediate event between an old mellow soul of the stall and all the propest and as a propention of the condition of the world in the stands of the condition of the every are is the stoll and all the every the propent of the condition of the condity of the man as the condition of the stall and the sains it is the stall and all the condition of the condition of the stall the stands in the still as the really the stands of the stall and the m------ temperature: 0.5
in the still as the really the stands of the stall and the most of the estaction of childer of all the stall to as a soul in the art and sympathy and manding as a peristing of the common and the wast in a same if stet in the stands of the
storder is to a whole
in the senses, and themselves 

KeyboardInterrupt: 

## Выводы:
    
1) Обучая модель для предсказания следующего токена по предшествующим, можно генерировать последовательности дискретных данных.

2) В случае с текстом такая модель называется языковой моделью; она может быть основана на словах или символах.

3) Выбор следующего токена требует баланса между мнением модели и случайностью.

4) Обеспечить такой баланс можно введением понятия температуры; всегда пробуйте разные температуры, чтобы найти правильную.